# Data Science dla studentów MBA Digital Transformation
## dr inż Marcin Luckner
### marcin.luckner@pw.edu.pl

## Cel zajęć

Celem zajęć jest zapoznanie uczestników z zagadnieniami klasyfikacji, regresji i klasteryzacji wykorzystując rzeczywiste dane. Do analizy danych użyty został język R (http://www.biecek.pl/R/#Pogromcy).

## Opis danych

Dane pochodzą z brytyjskiej agencji autostrad i zawierają rzeczywiste dane dotyczące przemieszczania się pojazdów pomiędzy skrzyżowaniami sieci autostrad.

### Struktura danych:

* #LinkRef - Unikalny identyfikator połączenia reprezentujący połączenie między skrzyżowaniami w sieci dróg zarządzanej przez Agencję Autostrad.
* #LinkDescription - Opis połączenia.
* #Date - data podróży.
* #TimePeriod - Jeden z 96 15-minutowych przedziałów w ciągu dnia, do których odnoszą się dane (0-95, gdzie 0 oznacza 00:00 do 00:15). 
* #AverageJT - Średni czas przejazdu przez połączenie w sekundach, dla pojazdów wjeżdżających na połączenie między skrzyżowaniami w danym 15-minutowym okresie czasu.
* #AverageSpeed - Średnia prędkość (w km/h) pojazdów wjeżdżających na połączenie między węzłami w danym 15-minutowym przedziale czasu.
* #DataQuality - Wskaźnik pokazujący jakość danych o czasie przejazdu dla danego połączenia i okresu czasu. 1 oznacza najwyższą jakość danych, a 5 - najniższą.
* #LinkLength -Długość połączenia (km).
* #Flow - Średnia z zaobserwowanego przepływu dla danego połączenia, okresu czasu i typu dnia. 

## Wczytanie danych:

Wczytanie pliku z danymi z lutego 2015 roku:

In [7]:
path = "./Data/"
filename = "FEB15.csv"
probe = read.csv(unzip(paste(path,filename,".zip",sep=''),filename))

Do przetwarzania danych korzystając z wygodnej notacji strumieniowej użyjemy bibliotek _plyr_ i _dplyr_ (http://genomicsclass.github.io/book/pages/dplyr_tutorial.html):

In [ ]:
  require(plyr)
  require(dplyr)

Usunięcie z danych zbędnej kolumny zawierającej opis i ograniczenie danych do rekordów o najlepszej jakości:

In [ ]:
  probe <- probe %>%
   filter(DataQuality == 1) %>%
   select(-LinkDescription)

Dodanie informacji o godzinie przejazdu:

In [ ]:
  probe$Hour = as.integer(probe$TimePeriod*15/60)

Dodanie informacji o dniu tygodnia przejazdu:

In [ ]:
  Sys.setlocale("LC_TIME", "en_US")
  probe$DayOfWeek = weekdays(as.Date(probe$Date))

  dow  = c("Monday", "Tuesday", "Wednesday", "Thursday","Friday", "Saturday", "Sunday")
  probe$DayOfWeek = factor(probe$DayOfWeek,levels=dow,ordered=TRUE)

Wczytane dane:

In [ ]:
head(probe)

## Eksploracja danych

Przegląd statystyk dla prędkości przejazu w poszczególnych dniach tygodnia:

In [ ]:
statistics <- probe %>% 
  group_by(DayOfWeek)%>%
  summarise(avg_speed = mean(AverageSpeed), 
            min_speed = min(AverageSpeed),
            max_speed = max(AverageSpeed),
            total = n())
View(statistics)

### Wizualizacje

W celu zaprezentowania danych użyjemy biblioteki _ggplot2_ (http://tutorials.iq.harvard.edu/R/Rgraphics/Rgraphics.html):

In [ ]:
require(ggplot2)

Rozkład średnich prędkości w zależności od dnia tygodnia:

In [ ]:
ggplot(probe, aes(DayOfWeek,AverageSpeed)) + geom_boxplot()+theme(text = element_text(size = 16))

In [ ]:
ggplot(probe, aes(AverageSpeed, fill=DayOfWeek))+geom_density(alpha=.3)+theme(text = element_text(size = 20))

Rozkład średnich prędkości w zależności od godziny:

In [ ]:
ggplot(probe, aes(Hour,AverageSpeed))+ geom_boxplot(aes(group = cut_width(Hour, 1)))+theme(text = element_text(size = 20))

## Klasteryzacja

Klasteryzacja dokonuje automatycznego podziału danych na klastry, czyli grupy podobnych do siebie rekordów.

Do klasteryzacji potrzebujemy narzędzi z biblioteki _e1071_:

In [ ]:
require('e1071')

Chcemy podzielić rekordy względem średniej prędkości _AverageSpeed_ na trzy grupy roboczo nazwane _low_, _medium_, i _high_. Ponieważ użyty mechanizm klasteryzacji, algorytm k-średnich (https://pl.wikipedia.org/wiki/Algorytm_centroidów), jest algorytmem iteracyjnym to określamy też liczbę iteracji.

In [ ]:
speeds = c('low','medium','high')
iterations <-10
columnName <-"AverageSpeed"

Algorytm k-średnich zwraca przypisanie każdej z obserwacji do klastra:

In [ ]:
k<-length(speeds)
clusters <- kmeans(probe[columnName], k, iterations);
classes <- clusters$cluster

Następnie porządkujemy uzyskane wyniki tak, aby robocza nazwa klastra odpowiadała zapisanym w nim wartościom:

In [ ]:
ord <-sort(clusters$centers, index.return=TRUE)

for(i in 1:k){
  classes[classes==i]  = speeds[ord$ix ==i]
}

probe$cluster = factor(classes,levels=speeds,ordered=TRUE)

Wykreślmy rozkład danych w stworzonych klastrach:

In [ ]:
ggplot(probe, aes(AverageSpeed,fill=cluster))+geom_density(alpha=.3)+theme(text = element_text(size = 20))

In [ ]:
ggplot(probe, aes(cluster,AverageSpeed))+ geom_boxplot()+theme(text = element_text(size = 20))

Wyliczmy ich statystyki:

In [ ]:
statistics <- probe %>% 
  group_by(cluster)%>%
  summarise(avg_speed = mean(AverageSpeed), 
            min_speed = min(AverageSpeed),
            max_speed = max(AverageSpeed),
            total = n())
View(statistics)

Klaster _low_ zawiera znacząco mniej obserwacji niż pozostałe klastry. Może to wpłynąć na dalszą analizę danych.

## Klasyfikacja

Klasyfikacja polega na przypisaniu obserwacji do jednej ze zdefiniowanych klas. Spróbujemy automatycznie przypisać przejazdy do jednego z trzech stworzonych klastrów.

### Drzewo decyzyjne
Do klasyfikacji użyjemy drzew decyzyjnych. Są to łatwe w interpretacji klasyfikatory, które zaimplementowano w bibliotece _rpart_, a ich wizualizację w bibliotece _rpart.plot_:

In [ ]:
require(rpart)
require(rpart.plot)

Spróbujemy określić do której kategorii przypisany zostanie przejazd na podstawie dnia tygodnia _DayOfWeek_, godziny _Hour_, i zatłoczenia _Flow_. Ponieważ zadanie to nie będzie łatwe, ograniczymy się do przewidywań dla pojedynczego odcinka autostrady _Link_.

In [ ]:
dependentVariable <-"cluster"
independentVariables <- c('DayOfWeek','Flow','Hour');
Link = 'AL2085'
part_probe<-probe %>%
  filter(LinkRef==Link)

Tworzymy formułę objaśniającą zmienną zależną od kombinacji zmiennych niezależnych:

In [ ]:
form<- paste(independentVariables, collapse = "+");
form <- as.formula(paste(dependentVariable, form, sep = " ~ "))
print(form)

Tworzymy drzewo decyzyjne:

In [ ]:
 probe.tree <- rpart(form, method='class', data=part_probe)

Stworzone drzewo prezentuje proste reguły pozwalające na określenie czy przejazd o danej godzinie będzie szybki (w zależności od zatłoczenia odcinka):

In [ ]:
rpart.plot(probe.tree)

Możemy zobaczyć jak wygląda proces uczenia drzewa w zależności od jego rozłożystości:

In [ ]:
printcp(probe.tree)

Biblioteka _caret_ zawiera mechanizmy oceny klasyfikatorów:

In [ ]:
require(caret)

Możemy teraz ocenić jakość predykcji dokonantch przez drzewo, wyliczając macierz pomyłek i pochodne dla niej miary jakości (https://en.wikipedia.org/wiki/Sensitivity_and_specificity):

In [ ]:
probe.tree.predictions <- predict(probe.tree ,part_probe, type = "class")
cm <- confusionMatrix(probe.tree.predictions, part_probe$cluster)
print(cm)

Nasze drzewo poprawnie zakwalifikowało 88 procent obserwacji, ale nie potrafi poprawnie rozpoznać obiektów z klasy _low_.

Wyświetlmy macierz pomyłek klasyfikacji w postaci graficznej:

In [ ]:
plt <- as.data.frame(cm$table)
plt$Prediction <- factor(plt$Prediction, levels=rev(levels(plt$Prediction)))

ggplot(plt, aes(Prediction,Reference, fill= Freq)) +
        geom_tile() + geom_text(aes(label=Freq),  size=10) +
        scale_fill_gradient(low="white", high="#009194") +
        labs(x = "Reference",y = "Prediction")+
        theme(text = element_text(size = 30), legend.position="none")       


### Testowanie drzewa

Drzewo było uczone i testowane na tych samych danych, pochodzących z lutego 2015. Sprawdzmy, czy działa równie dobrze na danych z innego okresu.

Wczytamy dane z marca 2015 roku i przypiszmy je do klastrów zgodnie z wcześniej ustalonymi regułami:

In [8]:
path = "./Data/"
filename = "MAR15.csv"
probe2 = read.csv(unzip(paste(path,filename,".zip",sep=''),filename))
probe2 <- probe2 %>%
   filter(DataQuality == 1) %>%
   filter(!is.na(Flow))%>%
   select(-LinkDescription)
probe2$DayOfWeek = weekdays(as.Date(probe2$Date))
probe2$DayOfWeek = factor(probe2$DayOfWeek,levels=dow,ordered=TRUE)
probe2$Hour = as.integer(probe2$TimePeriod*15/60)

#Labbeling of the testing set using rules from the first probe clusterisation
for(i in 1:length(speeds)){
 if(i==1){
   probe2$cluster[probe2$AverageSpeed<=statistics$max_speed[i]] = speeds[i]
 }else
    if(i==length(speeds)){
    probe2$cluster[ probe2$AverageSpeed>=statistics$min_speed[i]] = speeds[i]
    }else{
    probe2$cluster[probe2$AverageSpeed<=statistics$max_speed[i] & probe2$AverageSpeed>=statistics$min_speed[i]] = speeds[i]
  }
}
probe2$cluster = factor(probe2$cluster,levels=speeds,ordered=TRUE)


ERROR: Error in probe2 %>% filter(DataQuality == 1) %>% filter(!is.na(Flow)) %>% : nie udało się znaleźć funkcji '%>%'


In [ ]:
head(probe2)

Sprawdzmy, jak wcześniej utworzone drzewo sprawdzi się na nowych danych:

In [ ]:
dependentVariable <-"cluster"
independentVariables <- c('DayOfWeek','Flow','Hour')

part_probe2<-probe2 %>%
  filter(LinkRef==Link)
probe2.tree.predictions <- predict(probe.tree ,part_probe2, type = "class")
confusionMatrix(probe2.tree.predictions, part_probe2$cluster)

Drzewo uzyskało skuteczność 85 procent, niewiele mniejszą niż dla wcześniejszych danych. Oznacza to, że posiada ono zdolność generalizacji, czyli poprawnego działania dla nowych danych.

## Regresja

Zadanie regresji polega na przypisaniu do obserwacji pewnej zmiennej ciągłej.

### Las losowy

Las losowy jest zbiorem drzew, które w procesie głosowania wybierają wspólne rozwiązanie danego problemu. Decyzje lasu nie są tak łatwo interpretowalne jak decyzje pojedynczego drzewa, ale jego możliwości są znacznie większe. Las losowy jest zaimplementowany w bibliotece _randomForest_.

In [ ]:
 require(randomForest)

Chcemy estymować średnią prędkość _AverageSpeed_ na podstawie takich czynników jak dzień tygodnia _DayOfWeek_, natężenie ruchu _Flow_, godzina przejazdu _Hour_, i długość odcinka _LinkLength_.

In [ ]:
dependentVariable <-"AverageSpeed"
independentVariables <- c('DayOfWeek','Flow','Hour','LinkLength')

form<- paste(independentVariables, collapse = "+");
form <- as.formula(paste(dependentVariable, form, sep = " ~ "))
print(form)

Zbudujemy las złożony z 30 drzew losowych:

In [ ]:
nTree = 50
probe.rf <- randomForest(form,probe,ntree=nTree,importance=TRUE)


In [ ]:
probe.rf.predictions <- predict(probe.rf ,probe)
size = 500
df <- data.frame(
   y = c(probe$AverageSpeed[1:size], probe.rf.predictions[1:size]),
   x = c(1:size,1:size),
   probe = c(rep('speed', size), rep('estimation', size)) 
 )

In [ ]:
ggplot(df, aes(x, y, colour = probe, linetype=probe))+ geom_line(size=2)+theme(text = element_text(size = 30), legend.position="bottom")
ggsave("regression.pdf")

Wyliczmy średni błąd dla danych uczących:

In [ ]:
probe.rf.predictions <- predict(probe.rf ,probe)
probe.rf.errors <- abs(probe.rf.predictions-probe$AverageSpeed)
print(mean(probe.rf.errors))

Wyliczmy średni błąd dla danych testowych:

In [ ]:
probe2.rf.predictions <- predict(probe.rf ,probe2)
probe2.rf.errors <- abs(probe2.rf.predictions-probe2$AverageSpeed)
print(mean(probe2.rf.errors))

W celu porównania błędów dla obydwu zbiorów zestawimy je w jedną tabelę:

In [ ]:
df <- data.frame(
   x = c(probe.rf.errors, probe2.rf.errors),
   probe = c(rep('learning', length(probe.rf.errors)), rep('testing', length(probe2.rf.errors))) 
 )

Porównamy rozkład błędów stosując skumulowaną funkcję dystrybucji:

In [ ]:
 ggplot(df, aes(x, colour = probe)) + stat_ecdf(size=2)+theme(text = element_text(size = 20))

In [ ]:
median(probe2.rf.errors)

In [ ]:
median(probe.rf.errors)

Nie da się łatwo analizować reguł tworzonych przez las losowy, ale możemy analizować, które cechy były najważniejsze w kontekście minimalizacji błędu.

In [ ]:
importance(probe.rf)